In [22]:
import pandas as pd
import numpy as np
import scipy.optimize


dataset_path = 'dataset.csv'
df = pd.read_csv(dataset_path)


congruent_data = df['Congruent'].values
incongruent_data = df['Incongruent'].values

all_data = np.concatenate([congruent_data, incongruent_data])
conditions = np.concatenate([np.zeros_like(congruent_data), np.ones_like(incongruent_data)])

# Drift-diffusion model function
def ddm_model(params, t, condition):
    v, a, w, s = params
    drift = v if condition == 1 else -v  # Adjust drift based on condition
    noise = np.random.normal(0, abs(s), size=len(t))
    accumulation = np.cumsum(drift + noise)
    rt = np.argmax(np.abs(accumulation) >= a)
    return float(rt + w)

# Cost function for fitting
def cost_function(params, data, conditions):
    predicted_rts = np.array([ddm_model(params, np.arange(len(data)), c) for c in conditions])
    return np.sum((predicted_rts - data) ** 2)

# Initial parameter values
initial_parameters = np.array([0.34, 1.84, 17.56, 0.1])

# Fitting the model
fit_params = scipy.optimize.minimize(cost_function, initial_parameters, args=(all_data, conditions)).x

# Print the estimated parameters
print("Estimated Parameters:", fit_params)



# Modify new_conditions to contain 1000 alternating 0s and 1s
new_conditions = np.tile([0, 1], 1000)[:2000]

# Use the ddm_model function to predict reaction times for the updated conditions
predicted_rts = np.array([ddm_model(initial_parameters, np.arange(2000), c) for c in new_conditions])


print("Predicted Reaction Times:", predicted_rts)



Estimated Parameters: [ 0.34000028  1.84000625 17.56002952  0.1000332 ]
Predicted Reaction Times: [22.56 21.56 22.56 ... 21.56 23.56 22.56]


In [23]:
def count_odd_indexed_greater_than_previous(arr):
    count = 0
    for i in range(1, len(arr), 2):  # Start from index 1, step by 2 to get odd indices
        if arr[i] > arr[i - 1]:
            count += 1
    return count



result = count_odd_indexed_greater_than_previous(predicted_rts)

# Print the result
print("Number of cases where odd-indexed element is greater than its previous element:", result/1000)


Number of cases where odd-indexed element is greater than its previous element: 0.303


In [24]:
def average_odd_even(arr):
    odd_sum = 0
    even_sum = 0
    odd_count = 0
    even_count = 0

    for i in range(len(arr)):
        if i % 2 == 0:  # even index
            even_sum += arr[i]
            even_count += 1
        else:  # odd index
            odd_sum += arr[i]
            odd_count += 1

    if odd_count > 0:
        odd_average = odd_sum / odd_count
    else:
        odd_average = 0

    if even_count > 0:
        even_average = even_sum / even_count
    else:
        even_average = 0

    return odd_average, even_average

odd_avg, even_avg = average_odd_even(predicted_rts)
print("Average of Congruent:", odd_avg)
print("Average of Incongruent:", even_avg)


Average of Congruent: 22.487000000000233
Average of Incongruent: 22.532000000000238


In [20]:
import pandas as pd
from scipy.optimize import minimize
import numpy as np

# Load dataset
df = pd.read_csv("dataset.csv")

# Define drift diffusion model
def drift_diffusion_model(params, stimulus):
    """
    Simulate drift diffusion model for given parameters and stimulus
    
    Params:
    - params (tuple): Tuple of DDM parameters (drift rate, boundary separation, non-decision time)
    - stimulus (int): Stimulus indicating congruent (0) or incongruent (1) condition
    
    Returns:
    - response_time (float): Predicted response time
    """
    drift_rate, boundary, non_decision_time = params
    
    # Simulate accumulation process
    decision_variable = 0
    while abs(decision_variable) < boundary:
        decision_variable += drift_rate + np.random.normal(0, 1)  # Drift rate + noise
        decision_variable = max(0, decision_variable)  # Ensure non-negative
    
    # Add non-decision time
    response_time = non_decision_time + np.random.normal(0, 1)
    
    return response_time

# Define objective function for optimization
def objective(params):
    """
    Objective function for optimization
    
    Params:
    - params (tuple): Tuple of DDM parameters (drift rate, boundary separation, non-decision time)
    
    Returns:
    - error (float): Error between predicted and actual response times
    """
    error = 0
    for _, row in df.iterrows():
        congruent_rt = row['Congruent']
        incongruent_rt = row['Incongruent']
        
        # Predict response times
        predicted_congruent_rt = drift_diffusion_model(params, 0)
        predicted_incongruent_rt = drift_diffusion_model(params, 1)
        
        # Calculate error
        error += (predicted_congruent_rt - congruent_rt) ** 2
        error += (predicted_incongruent_rt - incongruent_rt) ** 2
    
    return error

# Initial guess for parameters
initial_params = (0.3, 1.5, 75)  # Example initial guess

# Perform optimization
result = minimize(objective, initial_params, method='Nelder-Mead')
best_params = result.x

print("Best parameters found:", best_params)


Best parameters found: [ 0.34181424  1.84558095 17.56124614]


In [4]:
import ddm